In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc
import base64
import io

# Load and prepare data
df = pd.read_csv("GDL-Subnational-HDI-data.csv")
df = df[df['Country'] == 'Canada']
df = df[df['Level'].isin(['National', 'Subnat'])]
year_columns = [str(y) for y in range(1990, 2023)]
df = df[['Region'] + year_columns]
df_long = df.melt(id_vars='Region', var_name='Year', value_name='SHDI')
df_long['Year'] = df_long['Year'].astype(int)

# Get list of unique regions
all_regions = sorted(df_long['Region'].unique())
default_regions = ['Total', 'Alberta', 'British Columbia', 'Ontario']

# Create heatmap as base64 image
heatmap_data = df_long.pivot(index='Region', columns='Year', values='SHDI')
fig, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(heatmap_data, annot=False, linewidths=0.5, cmap='YlGnBu', cbar_kws={'label': 'SHDI'})
plt.title('SHDI Heatmap (1990–2022)')
buf = io.BytesIO()
plt.savefig(buf, format='png')
plt.close(fig)
buf.seek(0)
encoded_heatmap = base64.b64encode(buf.read()).decode('utf-8')


In [ ]:
# Initialize app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Canada SHDI Dashboard'

# Layout with Tabs
app.layout = dbc.Container([
    html.H1('Canada Subnational Human Development Index (SHDI) Dashboard'),
    html.Hr(),
    dcc.Tabs([
        dcc.Tab(label='📊 SHDI Dashboard', children=[
            dbc.Row([
                dbc.Col([
                    html.Label('Select Provinces/Territories'),
                    dcc.Dropdown(
                        id='region-selector',
                        options=[{'label': r, 'value': r} for r in all_regions],
                        value=default_regions,
                        multi=True
                    )
                ], width=12)
            ]),
            html.Br(),
            dcc.Graph(id='trend-graph'),
            html.Br(),
            dcc.Graph(
                id='bar-chart',
                figure=px.bar(
                    df_long[df_long['Year'] == 2022].sort_values(by='SHDI', ascending=False),
                    x='Region',
                    y='SHDI',
                    title='SHDI Comparison by Region (2022)',
                    color='SHDI',
                    color_continuous_scale='viridis'
                )
            ),
            html.Br(),
            html.H4('🧭 SHDI Heatmap (1990–2022)'),
            html.Img(src='data:image/png;base64,{}'.format(encoded_heatmap), style={'width': '100%'})
        ]),
        dcc.Tab(label='📘 Documentation & Insights', children=[
            html.Br(),
            html.H4('SHDI Formula'),
            dcc.Markdown("""
            $$
            \text{SHDI} = \left( I_{\text{education}} \cdot I_{\text{health}} \cdot I_{\text{income}} \right)^{1/3}
            $$
            """),
            html.H4('Why SHDI Matters'),
            dcc.Markdown("""
            - **Education** reflects long-term investment in human capital.
            - **Health** indicates living conditions, nutrition, and health access.
            - **Income** proxies for economic capacity and access to goods/services.

            ### Interpretation
            - A SHDI closer to **1.000** means very high human development.
            - Regions above **0.900** are in the "very high" category.
            - SHDI helps identify inequities across regions and track progress over time.
            """)
        ])
    ])
], fluid=True)


In [ ]:
# Callbacks
@app.callback(
    Output('trend-graph', 'figure'),
    Input('region-selector', 'value')
)
def update_trend_chart(selected_regions):
    filtered_df = df_long[df_long['Region'].isin(selected_regions)]
    fig = px.line(
        filtered_df,
        x='Year',
        y='SHDI',
        color='Region',
        markers=True,
        title='SHDI Trends in Selected Regions (1990–2022)'
    )
    fig.update_layout(yaxis_range=[0.8, 1.0])
    return fig

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)
